In [4]:
import pandas as pd
df = pd.read_parquet("../data/processed/train_clean.parquet")

### 📊 EDAステップ1：ラベル分布（最重要）
Recall 重視のコンペでは、
陽性（judgement=1）の割合が戦略を決める最重要情報。

🔍 どう読むか
- judgement=1 が 多い → 閾値は低めでOK
- judgement=1 が 少ない → class_weight を強める必要あり
- judgement=1 が 中間 → 閾値調整が最重要

- **judgement=1 が 少ない:0.023282**

> これは 43件に1件しか陽性がないというレベル。
> つまり：
> - Recall を上げるのが極めて難しい
> - 閾値を下げる戦略は必須
> - class_weight も有効
> - ルールベースの Recall ブーストが強く効く
> β=7 の世界では、この不均衡はむしろ「戦略を明確にする材料」になる。


In [5]:
df["judgement"].value_counts(normalize=True).rename("ratio")

judgement
0    0.976718
1    0.023282
Name: ratio, dtype: float64

### 📏 EDAステップ2：タイトル・抄録の長さ分布
情報不足の1（judgement=1）が短文に偏っているかを見る。

judgment=1
```
title_len mean: 109.9
abstract_len mean: 1331.1
```

judgment=0
```
title_len mean: 102.6
abstract_len mean: 1250.9
```

読み取り
- judgement=1 の方が タイトルも abstract も長い
- 「情報不足の1」が多いと予想していたが、実際は 情報量が多い1が多い
これは重要で：
judgement=1 は“情報不足”ではなく、“診断精度研究っぽい長文”が多い

つまり：
- 長さフラグ（is_short_abstract）は効かない
- モデルは 長文の医学語彙を拾う方向で強く学習するべき


In [8]:
df["title_len"] = df["title"].str.len()
df["abstract_len"] = df["abstract"].str.len()
df[["title_len", "abstract_len"]].describe()

,title_len,abstract_len
count,27145.000000,27145.000000
mean,102.828882,1252.860858
std,37.227944,774.467059
min,16.000000,13.000000
25%,77.000000,761.000000
50%,99.000000,1359.000000
75%,124.000000,1745.000000
max,391.000000,9672.000000


In [10]:
df_judge1=df[df["judgement"]==1]
df_judge1["title_len"] = df_judge1["title"].str.len()
df_judge1["abstract_len"] = df_judge1["abstract"].str.len()
df_judge1[["title_len", "abstract_len"]].describe()

,title_len,abstract_len
count,632.000000,632.000000
mean,109.931962,1331.121835
std,36.257708,802.816944
min,16.000000,13.000000
25%,86.000000,791.750000
50%,106.000000,1555.000000
75%,132.000000,1826.250000
max,255.000000,4090.000000


In [9]:
df.groupby("judgement")[["title_len", "abstract_len"]].mean()

,title_len,abstract_len
judgement,,
0,102.659563,1250.995323
1,109.931962,1331.121835


### 🔤 EDAステップ3：clean_text の品質チェック
クリーニング後のテキストが壊れていないか確認する。

In [11]:
df["clean_text"].head(20).tolist()

['one-year age changes in mri brain volumes in older adults. longitudinal studies indicate that declines in cognition and memory accelerate after age 70 years. the neuroanatomic and neurophysiologic underpinnings of cognitive change are unclear as there is little information on longitudinal brain changes. we are conducting a longitudinal neuroimaging study of nondemented older participants in the baltimore longitudinal study of aging. this report focuses on age and sex differences in brain structure measured by magnetic resonance imaging during the first two annual evaluations. cross-sectional results from 116 participants aged 59-85 years reveal significantly larger ventricular volumes and smaller gray and white matter volumes in older compared with younger participants and in men compared with women. regional brain volumes show that the effects of age and sex are not uniform across brain regions. age differences are greatest for the parietal region. sex differences tend to be larger 

### 🔍 EDAステップ4：診断精度研究の典型キーワードの出現頻度
Recall ブースト用の辞書を作るための EDA。

読み取り
- ROC（19%）が圧倒的に多い
- 次に diagnostic（8.4%）
- sensitivity / specificity もそこそこ
これは：
診断精度研究の典型語彙が、データ全体にかなり含まれている

つまり：
- TF-IDF でも BERT でも 語彙ベースで分類しやすい構造
- ルールベース辞書が非常に効く（Recall ブースト）


In [12]:
keywords = [
    "sensitivity", "specificity", "roc", "auc",
    "diagnostic", "accuracy", "likelihood",
    "検査", "感度", "特異度", "診断"
]

for kw in keywords:
    rate = df["clean_text"].str.contains(kw, case=False).mean()
    print(f"{kw}: {rate:.3f}")

sensitivity: 0.058
specificity: 0.045
roc: 0.190
auc: 0.014
diagnostic: 0.084
accuracy: 0.036
likelihood: 0.008
検査: 0.000
感度: 0.000
特異度: 0.000
診断: 0.000


### 🔍 EDAステップ5：judgement=1 の“怪しい1”を可視化
このコンペ特有の重要ポイント。
judgement=1 には「診断精度研究じゃない1」が混ざる。


- COVID-19 関連が多い
- 感染症診断、画像診断、PCR、AI診断など
- 「診断精度研究」っぽいものが多い
- ただし “Environmental challenge.” のような 怪しい1 も混ざる
読み取り
- 陽性ラベル側にノイズはあるが、
大半は診断精度研究に近い内容
→ モデルは「診断精度研究の語彙」を学習すれば良い
→ ノイズはあるが、Recall 重視なら問題ない


In [13]:
df[df["judgement"] == 1].sample(20)[["title", "abstract"]]

,title,abstract
15211,SARS-CoV-2 infection serology validation of di...,BACKGROUND AND AIMS: A novel coronavirus (SARS...
14926,Assessment of Sensitivity and Specificity of P...,[NO_ABSTRACT]
18097,Prospective study in 355 patients with suspect...,OBJECTIVE: To evaluate the incidence of certai...
6977,Differentiating between visual hallucination-f...,[NO_ABSTRACT]
20696,What is the Preferred Screening Tool for COVID...,"INTRODUCTION: Since the outbreak of COVID-19, ..."
10191,Ultrasensitive and visual detection of SARS-Co...,The recent outbreak of novel coronavirus (SARS...
24175,Development and evaluation of an artificial in...,Early detection of COVID-19 based on chest CT ...
5685,The Model for Early COvid-19 Recognition (MECO...,This study aims to assess the peripheral blood...
14373,Relevance of ultrasonographic parameters in pr...,Background: Bowel ultrasound (B-US) has been w...
12911,Results of the first pilot external quality as...,Objectives Assessment of severe acute respirat...


### 🔍 EDAステップ6：TF-IDF の top n-grams（1 vs 0 比較）
TF-IDF を使う場合は必須。

judgement=1 の top 語彙
```
cov, sars, covid, 19, covid 19, pcr, detection, sensitivity, diagnosis, diagnostic, specificity, ct
```

judgement=0 の top 語彙
```
disease, ad, by, on, as, from, we
```

読み取り
- judgement=1 は 感染症診断・PCR・COVID・診断精度語彙が強い
- judgement=0 は **一般医学語彙（disease, by, on）**が多い
これは非常に良い傾向で：
TF-IDF + ロジットでも十分戦える構造

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vec = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = vec.fit_transform(df["clean_text"])

# numpy の bool 配列に変換
mask_1 = (df["judgement"] == 1).values
mask_0 = (df["judgement"] == 0).values

# ラベル別の平均TF-IDF値
mean_1 = X[mask_1].mean(axis=0)
mean_0 = X[mask_0].mean(axis=0)

# 上位語彙を抽出
vocab = np.array(vec.get_feature_names_out())

top_1 = vocab[np.argsort(-mean_1.A1)[:30]]
top_0 = vocab[np.argsort(-mean_0.A1)[:30]]

top_1, top_0

(array(['the', 'of', 'and', 'cov', 'sars', 'in', 'sars cov', 'covid', '19',
        'covid 19', 'for', 'patients', 'to', 'with', 'no abstract',
        'abstract', 'dlb', 'pcr', 'detection', 'sensitivity', 'were', 'no',
        'assay', 'samples', 'rt', 'was', 'diagnosis', 'diagnostic',
        'specificity', 'ct'], dtype=object),
 array(['the', 'of', 'and', 'in', 'to', 'with', 'covid', '19', 'covid 19',
        'patients', 'no abstract', 'abstract', 'no', 'for', 'was', 'were',
        'of the', 'is', 'disease', 'in the', 'that', 'ad', 'by', 'on',
        'as', 'sars', 'we', 'cov', 'sars cov', 'from'], dtype=object))

### 🎯 EDAステップ7：clean_text の長さとラベルの関係

読み取り
- judgement=1 の方が 語彙が多い
- クリーニングで情報が落ちていない
- モデルは長文の語彙を拾いやすい


In [15]:
df["clean_len"] = df["clean_text"].str.split().str.len()
df.groupby("judgement")["clean_len"].describe()

,count,mean,std,min,25%,50%,75%,max
judgement,,,,,,,,
0,26513.0,197.393958,115.540346,4.0,123.0,213.0,271.0,1546.0
1,632.0,212.523734,121.169222,4.0,125.0,246.5,289.0,629.0


🎯 総合結論：このデータの“構造”はこうなっている
- 陽性率 2.3% → 超不均衡 → Recall が最重要
- judgement=1 は 長文＋診断精度語彙が多い
- 日本語はほぼ存在しない → 英語中心でOK
- ROC / diagnostic / sensitivity / specificity が強い特徴語
- TF-IDF でも BERT でも分類しやすい構造
- judgement=1 にノイズはあるが、Recall 重視なら問題なし
- ルールベース辞書が強く効く（特に ROC）
